## Image Denosing with Sauvola's Algorithm


In [42]:
import skimage
from skimage.filters import threshold_sauvola

import numpy as np
import os

import matplotlib.pyplot as plt

In [43]:
# Get Noise Images File
def get_image_files(noise_images_dir: str)-> list[str]:
    image_files = []
    image_extensions = ('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff', '.webp')
    for root, _, files in os.walk(noise_images_dir):
        temp_image_files = [
            root+r'/'+file_name 
            for file_name in files 
            if file_name.lower().endswith(
                image_extensions
            )
        ]
        image_files = [*image_files,*temp_image_files]
    return image_files

In [62]:
def read_image(image: str)-> np.ndarray:
    image = skimage.io.imread(image)
    return image

# TO Do -> need to define window_size by image
def binarize_with_sauvola(image: np.ndarray, window_size: int=25)-> np.ndarray:
    thresh_sauvola = threshold_sauvola(image, window_size=window_size)
    binary_sauvola = image > thresh_sauvola
    return binary_sauvola

In [66]:
def check_dir_exist(dir_name: str, create_dir: bool = False)-> None:
    path_exists = os.path.exists(dir_name)
    if not path_exists:
        print(f"{dir_name} does not exist.")
        if create_dir:
            print(f"Creating {dir_name} directory.")
            # dir create

In [63]:
def get_binary_images(
        images_dir: str, 
        save_dir: str, 
        window_size: int=25,
    ) -> None:
    check_dir_exist(dir_name=images_dir)
    check_dir_exist(dir_name=save_dir, create_dir=True)
    image_files = get_image_files(images_dir)
    for image_name in image_files:
        image = read_image(image_name)
        binary_image = binarize_with_sauvola(image)
        # fig, (ax1, ax2) = plt.subplots(1, 2)
        # ax1.imshow(image)
        # ax1.set_title('Original Image')
        # ax2.imshow(binary_image)
        # ax2.set_title('Binarize Image with Sauvola')
        plt.imshow(binary_image, cmap=plt.cm.gray)
        plt.show()

In [68]:
get_binary_images("noise_images", "")

 does not exist.
Creating  directory.


ValueError: Unsupported dtype

<Figure size 640x480 with 1 Axes>

ValueError: Unsupported dtype

<Figure size 640x480 with 1 Axes>

ValueError: Unsupported dtype

<Figure size 640x480 with 1 Axes>